# 1. 크롤러 스킵

- 필요시 추가예정

# 2. 뉴스,주가 데이터 임포트
## 형태소 분석 ~ 전처리 ~ 익스포트

In [11]:
import pandas as pd
import numpy as np
import re
import datetime
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
from konlpy.tag import Okt
okt = Okt()

#from eunjeon import Mecab
#mecab = Mecab()

# class 정의 => 기능은 뉴스랑 주가데이터 임포트 ~ 전처리데이터 익스포트
# filename : "news_all.csv"
# filepath : "./dataset/"
class data_import_preprocess():
    def news_get(self, filepath):
        df = pd.read_csv(filepath, encoding = 'utf-8 sig', engine = 'python')
        return df

    # 두번째 메소드 : news데이터 전처리 밑 명사로 나눠주는 함수
    # 1param - InputData
    # 칼럼 이름은 고정값(아래 함수에서 지정한 칼럼이름만 처리 가능)
    def news_timecleansing(self, data):
        # Time 칼럼의 노이즈 제거
        df_news = data
        df_news['TIME'] = df_news['TIME'].apply(lambda x: x[-14:]) # 연도 앞에 붙은 문자열 제거 (뒤에서 14개문자만 자르기)
        df_news['TIME'] = df_news['TIME'].apply(lambda x: x[:10]) # 분과 초 제거 (앞에서 10개 문자만 자르기)
        df_news['TIME'] = pd.to_datetime(df_news['TIME'])# 표준 날짜타입으로 변환
        return df_news
    
    # 형태소분석기로 명사만 추출하기
    # 1param - InputData / 2param - 사용할 형태소 분석기
    def noun_extracting(self, df, extractor):        
        df['title+body'] = df.apply(lambda x : '{}|||{}'.format(x['TITLE'], x['BODY']), axis = 1) # title과 body 합쳐서 새로운 칼럼 추가  구분자 : |||
        df['title+body'] = df['title+body'].astype(str)
        df['NOUNS'] = df['title+body'].apply(lambda x : '{}'.format(extractor.nouns(x))) # ''씌워줘야 list 안 원소들이 join해서 붙는다.
        return df

    # 1param - 종목코드 / 2param - startdate / 3param - enddate
    def stock_get(self, keycode, startdate, enddate):
        yf.pdr_override()
        df_stock = pdr.get_data_yahoo(keycode, start = startdate, end = enddate)
        df_stock['day1T'] = df_stock['Close'].shift(-1)
        df_stock['day3T'] = df_stock['Close'].shift(-3)
        df_stock['day5T'] = df_stock['Close'].shift(-5)
        df_stock['day10T'] = df_stock['Close'].shift(-10)
        return df_stock
    
    def news_stock_concat(self, news, stock):
        news.rename(columns={'TIME': 'Date'}, inplace = True)
        news = news.groupby('Date').agg({'NOUNS' : lambda x : ''.join(x)}) 
        news_stock = news.join(df_stock, how = 'outer')
        news_stock = news_stock.dropna()
        return news_stock
        
    def making_new_columns(self, df):
        df['Diff1'] = df['day1T'] - df['Close']
        df['Diff3'] = df['day3T'] - df['Close']
        df['Diff5'] = df['day5T'] - df['Close']
        df['Diff10'] = df['day10T'] - df['Close']
        df['Diff1_per'] = round(df['Diff1'] / df['Close'] * 100, 2)
        df['Diff3_per'] = round(df['Diff3'] / df['Close'] * 100, 2)
        df['Diff5_per'] = round(df['Diff5'] / df['Close'] * 100, 2)
        df['Diff10_per'] = round(df['Diff10'] / df['Close'] * 100, 2)
        bins = [-100, 0, 100]
        labels = [-1, 1]
        df['Diff1_clf'] = pd.cut(df["Diff1_per"], bins, labels = labels)
        df['Diff3_clf'] = pd.cut(df["Diff3_per"], bins, labels = labels)
        df['Diff5_clf'] = pd.cut(df["Diff5_per"], bins, labels = labels)
        df['Diff10_clf'] = pd.cut(df["Diff10_per"], bins, labels = labels)
        df.drop(['High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace = True)
        return df

# 클래스 내 메소드 순차적 실행하는 예제코드

In [2]:
# 인스턴스 생성
cls = data_import_preprocess()

df_news = cls.news_get('./Dataset/삼성전자.csv')
df_news = cls.news_timecleansing(df_news)
df_news = cls.noun_extracting(df_news, okt)

df_stock = cls.stock_get("005930.KS", "2017-12-01", "2019-01-08")

df_news_stock = cls.news_stock_concat(df_news, df_stock)
df_news_stock_new = cls.making_new_columns(df_news_stock)
df_news_stock_new.head()

[*********************100%***********************]  1 of 1 downloaded


,NOUNS,Open,Close,day1T,day3T,day5T,day10T,Diff1,Diff3,Diff5,Diff10,Diff1_per,Diff3_per,Diff5_per,Diff10_per,Diff1_clf,Diff3_clf,Diff5_clf,Diff10_clf
Date,,,,,,,,,,,,,,,,,,,
2017-12-01,"['내년', '증시', '정기', '대비', '자산', '배분', '펀드', '주목...",50800.0,50840.0,51340.0,50020.0,52000.0,50620.0,500.0,-820.0,1160.0,-220.0,0.98,-1.61,2.28,-0.43,1,-1,1,-1
2017-12-04,"['투자자', '항의', '빗발', '친', '모간', '스탠리', '삼성', '전...",50840.0,51340.0,51260.0,50740.0,51780.0,51200.0,-80.0,-600.0,440.0,-140.0,-0.16,-1.17,0.86,-0.27,-1,-1,1,-1
2017-12-05,"['이', '종목', '왜', '오늘', '특징', '주', '내년', '기대', ...",50600.0,51260.0,50020.0,52000.0,52100.0,51560.0,-1240.0,740.0,840.0,300.0,-2.42,1.44,1.64,0.59,-1,1,1,1
2017-12-06,"['서울대', '시흥', '캠', '삼성', '현대차', '자율', '주행', '연...",51260.0,50020.0,50740.0,51780.0,51320.0,49140.0,720.0,1760.0,1300.0,-880.0,1.44,3.52,2.60,-1.76,1,1,1,-1
2017-12-07,"['과', '뇌물', '공범', '방', '특검', '최', '실', '공모', '...",50040.0,50740.0,52000.0,52100.0,51060.0,49700.0,1260.0,1360.0,320.0,-1040.0,2.48,2.68,0.63,-2.05,1,1,1,-1


# 최종결과물 확인
- 3만 넘으면 Ok

In [3]:
len(df_news_stock_new)
len(df_news_stock_new.iloc[0,0])

36402

# 3-1. 통계분류로 예측(SVC, NaiveBaysian)
## Vectorization => countervec + tf/idf로 학습

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from gensim.models.doc2vec import Doc2Vec
count_vect = CountVectorizer()
TfidfTransformer()

# 파이프라인 쓰는법도 고려해보자!!!! 더 쉽고 간결함
# 시각화 함수도 넣으면 좋을듯? 
class Vector():
    
    def CounterVec_tfidf(self, df, n, k, ratio):
        # DataFrame에서 필요한 칼럼추출 => split
        X = df['NOUNS']
        y = df.loc[:, ['Diff' + n + '_clf']]
        
        # 카운터벡터
        X_vec = count_vect.fit_transform(X)
        
        # tfidf 가중치 적용
        tfidf_transformer = TfidfTransformer(use_idf = True).fit(X_vec) 
        X_vec_tfidf = tfidf_transformer.transform(X_vec)
        
        # chi2 변수선택
        X_vec_tfidf_chi2 = SelectKBest(chi2, k = k).fit_transform(X_vec_tfidf, y)

        # train / test 나누기
        X_train, X_test, y_train, y_test = train_test_split(X_vec_tfidf_chi2, y, test_size = ratio)
        
        return X_train, X_test, y_train, y_test, tfidf_transformer
    

    # import할 모델이름 => doc2vec_no_pos_tagging.model
    def Doc_model_import(self, filepath):
        model_wikinews = Doc2Vec.load(filepath)
        return model_wikinews
    

    def Doc_vector(self, model, df, n, ratio):
        model_wikinews = model
        
        X = df['NOUNS']
        y = df.loc[:, ['Diff' + n + '_clf']]
        
        
        X_corpus_vectors = []
        for i in X:
            infer_vectors = model_wikinews.infer_vector(i)
            X_corpus_vectors.append(infer_vectors)
        print('뽑아낸 Vector 크기 : {}'.format(len(X_corpus_vectors)))
            
        X_train, X_test, y_train, y_test = train_test_split(X_corpus_vectors, y, test_size = ratio)
        
        return X_train, X_test, y_train, y_test
        
        
# 일단 상속은 받아놓고 시작
class classification(Vector): 
    
    def NB_tfidf(self, X_train_vec, X_test_vec, y_train, y_test):
        MultinomialNB(alpha = 1.0, class_prior = None, fit_prior = True)
        clf_nb = MultinomialNB()
        clf_nb.fit(X_train_vec, y_train)
        
        #predicted = clf_tfidf.predict(X_test_vec)
        #print('tfidf로 변환한 예측률 :',np.mean(predicted == y_test))
        
        print('훈련 셋 검증결과 : {:.3f}.'.format(clf_nb.score(X_train, y_train)))
        print('테스트 셋 검증결과 : {:.3f}.'.format(clf_nb.score(X_test, y_test)))
        #print('UP/Down 예측값 : {:.3f}.'.format(clf_nb.predict_proba(X_test)))  만드는 중
        print(clf_nb.predict(X_train))
        

    # [과제]gamma는 0.1 / c는 10 디폴트로 설정해보자
    # 1param - Xdata / 2param - ydata / 3param - ratio / 4param - N일 뒤 예측(1,3,5,10)
    def SVC_tfidf(self, X_train_vec, X_test_vec, y_train, y_test):  # ratio - 0.2 / gamma - 0.1 / c - 10 디폴트하거나 파라미터 삭제
        clf_svc = SVC(gamma = 0.1, class_weight = "balanced", kernel = 'rbf', C = 10)
        clf_svc.fit(X_train_vec, y_train)
        
        print('훈련 셋 검증결과 : {:.3f}.'.format(clf_svc.score(X_train_vec, y_train)))
        print('테스트 셋 검증결과 : {:.3f}.'.format(clf_svc.score(X_test_vec, y_test)))
        #print(clf_svc.predict_proba(X_test))
        print(clf_svc.predict(X_train))
    
    
        # 퍼센트 안내멘트를 리턴하면 될듯?

## 테스트 실행1 - countVec + Naive Baysian 분류기
- 한글설명 : https://datascienceschool.net/view-notebook/c19b48e3c7b048668f2bb0a113bd25f7/

In [13]:
# 이전 모듈 결과값 : df_news_stock_new
vec = Vector()
clf = classification()

X_train, X_test, y_train, y_test, tfidf_transformer = vec.CounterVec_tfidf(df_news_stock_new, n = '1', k = 20000, ratio = 0.2)
nb = clf.NB_tfidf(X_train, X_test, y_train, y_test)

훈련 셋 검증결과 : 0.639.
테스트 셋 검증결과 : 0.490.
[-1  1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1
 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1  1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1  1 -1 -1 -1 -1 -1  1 -1 -1
 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1  1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1  1 -1 -1 -1  1 -1  1]


C:\Users\sbg02\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## 테스트 실행2 - countVec + Kernel SVM 분류기
- 한글설명 : https://tensorflow.blog/파이썬-머신러닝/2-3-7-커널-서포트-벡터-머신/

In [14]:
vec = Vector()
clf = classification()

X_train, X_test, y_train, y_test = vec.CounterVec_tfidf(df_news_stock_new, n = '1', k = 10000, ratio = 0.2)
nb = clf.SVC_tfidf(X_train, X_test, y_train, y_test)

C:\Users\sbg02\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


훈련 셋 검증결과 : 0.965.
테스트 셋 검증결과 : 0.745.
[ 1  1 -1 -1 -1  1 -1 -1  1  1 -1 -1 -1 -1  1  1  1 -1  1 -1 -1  1  1  1
 -1  1  1 -1 -1  1  1 -1  1  1 -1 -1  1 -1 -1 -1 -1 -1  1  1 -1  1 -1 -1
 -1  1  1 -1 -1 -1 -1  1 -1  1 -1  1  1  1  1 -1  1  1 -1  1 -1 -1 -1  1
  1  1 -1 -1 -1 -1  1  1  1 -1 -1  1 -1 -1 -1 -1 -1 -1  1  1  1  1 -1  1
 -1 -1 -1 -1  1 -1 -1 -1  1  1  1  1  1 -1  1  1  1 -1  1 -1 -1  1  1  1
 -1 -1  1 -1 -1  1  1  1 -1 -1 -1  1  1  1  1  1 -1  1 -1  1  1 -1 -1 -1
 -1 -1  1 -1  1  1  1 -1 -1 -1  1 -1  1 -1  1  1 -1 -1 -1  1  1 -1  1  1
 -1  1  1  1  1 -1 -1 -1  1 -1  1 -1  1  1 -1 -1 -1  1 -1  1 -1 -1  1 -1
  1  1 -1  1  1 -1 -1  1 -1  1]


# 3-2. 통계분류로 예측(svc, NaiveBaysian)
## Vectorization => Doc2Vec으로 학습

## 테스트 실행3 - Doc2Vec + Kernel SVM 분류기

In [7]:
vec = Vector()
clf = classification()

# 모델 임포트하는데 시간 좀 걸립니다. ㅠ / 모델 합치면 6G정도, 웬만한 노트북은 디스크 긁으면서 돌아갑니다.
model = vec.Doc_model_import('./Doc2VecClf/doc2vec_no_pos_tagging.model')

# 함수에 모델 넣는것도 오래걸린다 ㅠ
X_train, X_test, y_train, y_test = vec.Doc_vector(model, df_news_stock_new, n = '5', ratio = 0.2)

뽑아낸 Vector 크기 : 253


In [8]:
X_train, X_test, y_train, y_test = vec.Doc_vector(model, df_news_stock_new, n = '10', ratio = 0.2)

뽑아낸 Vector 크기 : 253


In [9]:
vec = Vector()
clf = classification()

svc = clf.SVC_tfidf(X_train, X_test, y_train, y_test)

훈련 셋 검증결과 : 1.000.
테스트 셋 검증결과 : 0.627.
[-1 -1  1  1  1  1 -1  1 -1 -1 -1 -1 -1  1  1 -1  1  1 -1 -1  1 -1  1 -1
 -1 -1  1 -1 -1 -1  1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1  1 -1 -1
 -1  1 -1 -1 -1  1 -1 -1  1 -1 -1 -1  1 -1 -1 -1 -1 -1  1  1  1 -1  1 -1
  1  1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1  1  1 -1 -1 -1  1 -1  1 -1
  1  1 -1 -1  1 -1  1  1 -1 -1 -1 -1 -1 -1  1  1  1  1 -1  1  1  1  1 -1
 -1  1  1 -1 -1 -1 -1 -1  1 -1  1 -1  1 -1 -1  1 -1 -1 -1  1 -1 -1 -1 -1
  1 -1  1 -1  1 -1 -1 -1  1 -1  1 -1 -1  1 -1 -1 -1  1 -1  1 -1 -1  1 -1
 -1  1 -1  1  1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1  1
 -1  1 -1 -1  1  1 -1  1 -1 -1]


C:\Users\sbg02\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
!pip show gensim

Name: gensim
Version: 3.7.0
Summary: Python framework for fast Vector Space Modelling
Home-page: http://radimrehurek.com/gensim
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPLv2.1
Location: c:\users\sbg02\anaconda3\lib\site-packages
Requires: six, smart-open, numpy, scipy
Required-by: 


## 테스트 실행4 - Doc2Vec + Naive Baysian 분류기
- 코드는 이상 없는데, doc2vec에서 벡터를 추출하면 음수값이 섞여있으서 분류기 학습이 안된다.ㅠ

In [ ]:
vec = Vector()
clf = classification()

svc = clf.NB_tfidf(X_train, X_test, y_train, y_test)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
#count_vect = CountVectorizer()


# 파이프라인 쓰는법도 고려해보자!!!! 더 쉽고 간결함
# 시각화 함수도 넣으면 좋읗듯? 
class VectorFeature():
    
    def split_vectorize(self, df, n, ratio):
        # DataFrame에서 필요한 칼럼추출 => split
        X = df['NOUNS']
        y = df.loc[:, ['Diff'+n+'_clf']]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = ratio)
        
        
        # 벡터화+tf/idf 해주는 pipeline 설계
        makevector = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
        ])
        
                
        # 벡터화 진행 후 저장(X_data만  실시)
        X_train_vec = makevector(X_train)
        X_test_vec = makevector(X_test)
        return X_train_vec, X_test_vec, y_train, y_test
    
    
    # k =15000정도 괜춘
    def feature_Select(self, X_train_vec, X_test_vec, y_train, y_test, k):
        # X벡터를 y값과 비교해 카이제곱검정으로 벡터 축소
        X_train_chi2 = SelectKBest(chi2, k = k).fit_transform(X_train_vec, y_train)
        X_test_chi2 = SelectKBest(chi2, k = k).fit_transform(X_test_vec, y_test)
        print(X_train_chi2.shape)
        print(X_test_chi2.shape)
        return X_train_chi2, X_test_chi2
    
    
''' 위 함수에 메소드 오버라이딩해보면 좋을 듯?
    super().feature_Select(X_train_vec, y,  k)   이렇게...?


    # k =15000정도 괜춘
    def feature_Select(self, X_train_vec, X_test_vec, y_train, y_test, k):
        # X벡터를 y값과 비교해 카이제곱검정으로 벡터 축소
        X_train_chi2 = SelectKBest(chi2, k = k).fit_transform(X_train_vec, y_train)
        X_test_chi2 = SelectKBest(chi2, k = k).fit_transform(X_test_vec, y_test)
        print(X_train_chi2.shape)
        print(X_test_chi2.shape)
        return X_train_chi2, X_test_chi2
'''
    
class classification(VectorFeature): 
    
    def NB_tfidf(self, X_train_vec, X_test_vec, y_train, y_test):
        MultinomialNB(alpha = 1.0, class_prior = None, fit_prior = True)
        clf_nb = MultinomialNB()
        clf_nb.fit(X_train_vec, y_train)
        
        #predicted = clf_tfidf.predict(X_test_vec)
        #print('tfidf로 변환한 예측률 :',np.mean(predicted == y_test))
        
        print('훈련 셋 검증결과 : {:.3f}.'.format(clf_nb.score(X_train, y_train)))
        print('테스트 셋 검증결과 : {:.3f}.'.format(clf_nb.score(X_test, y_test)))
        #print('UP/Down 예측값 : {:.3f}.'.format(clf_nb.predict_proba(X_test)))  만드는 중
        print(clf_nb.predict(X_train))
        

    # [과제]gamma는 0.1 / c는 10 디폴트로 설정해보자
    # 1param - Xdata / 2param - ydata / 3param - ratio / 4param - N일 뒤 예측(1,3,5,10)
    def SVC_tfidf(self, X_train_vec, X_test_vec, y_train, y_test):  # ratio - 0.2 / gamma - 0.1 / c - 10 디폴트하거나 파라미터 삭제
        clf_svc = SVC(gamma = 0.1, class_weight = "balanced", kernel = 'rbf', C = 10)
        clf_svc.fit(X_train_vec, y_train)
        
        print('훈련 셋 검증결과 : {:.3f}.'.format(clf_svc.score(X_train_vec, y_train)))
        print('테스트 셋 검증결과 : {:.3f}.'.format(clf_svc.score(X_test_vec, y_test)))
        #print(clf_svc.predict_proba(X_test))
        print(clf_svc.predict(X_train))
    
    
        # 퍼센트 안내멘트를 리턴하면 될듯?

In [ ]:
    # param1 - data / param2 - testSet 비율
    # X_train과 X_test만 벡터화시키면 된다. y는 그냥 칼럼 슬라이싱만 하면 ok(아직은 y값 2진분류)
    def split_vectorize(df, ratio):        
        # Train data 벡터 + tf/idf
        X_train_counts = count_vect.fit_transform(X) 
        tfidf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts) 
        X_train_tfidf = tfidf_transformer.transform(X_train_counts)
        
        # Test data 벡터 + tf/idf
        X_test_counts = count_vect.transform(X)
        X_test_tfidf = tfidf_transformer.transform(X_test_counts)

        return (X_train_tfidf, X_test_tfidf, y_train, y_test)
        
    # 아직 y슬라이싱 안하고 집어넣음 =-> 수정 보완 ㄱㄱㄱㄱㄱㄱㄱㄱㄱㄱ 
    def NB_tfidf(df):
        clf_train = MultinomialNB().fit(X_train_tfidf, y_train)
        predicted_train = clf_tfidf.predict(X_train_tfidf)
        print('======== Train데이터 예측률 ========')
        print('tfidf로 변환한 예측률 :', np.mean(predicted_train == y_train))
        
        clf_test = MultinomialNB().fit(X_test_tfidf, y_test)
        print('======== Test데이터 예측률 ========')
        print('tfidf로 변환한 예측률 :', np.mean(clf_test == y_test))
        return 뭘 뽑아야 할까요`~?